# Titanic

In [1]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv('data/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [139]:
train[train.Sex == 'female'].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## Features

In [127]:
from sklearn.preprocessing import FunctionTransformer

y_train = train['Survived']

attributes = ['Pclass', 'Age', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']
fs = FunctionTransformer(lambda X: X[attributes])

X_train = fs.fit_transform(train)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Age       714 non-null    float64
 2   Sex       891 non-null    object 
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [128]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

median_imp = SimpleImputer(strategy='median')

embarked_imp = SimpleImputer(strategy='most_frequent')
embarked_encoder = OneHotEncoder()
embarked_pipe = Pipeline([
    ('fill', embarked_imp),
    ('encode', embarked_encoder)
])



ct = ColumnTransformer(remainder='passthrough',
    transformers = [
        ('age', median_imp, ['Age', 'Fare']),
        ('embarked', embarked_pipe, ['Embarked']),
        ('sex', OrdinalEncoder(), ['Sex'])
    ])

full_pipeline = Pipeline([
    ('feature_selector', fs),
    ('column_transform', ct)
])

In [129]:
X_train = full_pipeline.fit_transform(train)
X_train[: 5]

array([[22.    ,  7.25  ,  0.    ,  0.    ,  1.    ,  1.    ,  3.    ,
         1.    ,  0.    ],
       [38.    , 71.2833,  1.    ,  0.    ,  0.    ,  0.    ,  1.    ,
         1.    ,  0.    ],
       [26.    ,  7.925 ,  0.    ,  0.    ,  1.    ,  0.    ,  3.    ,
         0.    ,  0.    ],
       [35.    , 53.1   ,  0.    ,  0.    ,  1.    ,  0.    ,  1.    ,
         1.    ,  0.    ],
       [35.    ,  8.05  ,  0.    ,  0.    ,  1.    ,  1.    ,  3.    ,
         0.    ,  0.    ]])

## Models

In [106]:
class Trainer:
    def __init__(self, X_train, y_train, cv=5):
        self.X = X_train
        self.y = y_train
        self.cv = cv

    def get_best_model(self, model, search_grid):
        grid_search = GridSearchCV(model, params_grid, cv=self.cv,
                               scoring='accuracy',
                              n_jobs=-1, verbose=2)
        grid_search.fit(self.X, self.y)
        print(f'Best score: {grid_search.best_score_}')
        print(f'Configuration: {grid_search.best_params_}')
        return grid_search.best_estimator_
    
    
trainer = Trainer(X_train, y_train)

### Random forrest

In [107]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf_clf = RandomForestClassifier()
params_grid = [
    {'n_estimators': [3, 10, 30, 50], 'max_features': [3, 4, 5, 6, 7, 8, 9, 10]}
]
rf_best = trainer.get_best_model(rf_clf, params_grid)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    1.2s


Best score: 0.8170736300295023
Configuration: {'max_features': 6, 'n_estimators': 30}


[Parallel(n_jobs=-1)]: Done 153 out of 160 | elapsed:    2.7s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:    2.9s finished


In [140]:
rf_best.feature_importances_

array([0.24792415, 0.25424586, 0.01170507, 0.00377201, 0.01309369,
       0.3000325 , 0.0995463 , 0.03983463, 0.0298458 ])

### SVC

In [94]:
from sklearn.svm import SVC
svm_clf = SVC()

params_grid = [
    {'kernel': ['linear', 'poly', 'rbf'], 'C': [0.1, 1, 10, 50]}
]
svm_best = trainer.get_best_model(svm_clf, params_grid)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  2.3min finished


Best score: 0.7878601468834348
Configuration: {'C': 0.1, 'kernel': 'linear'}


### KNeighbors

In [95]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()

params_grid = [
    {'weights': ['distance', 'uniform'], 'n_neighbors': [2, 3, 4, 5, 6, 7, 10, 20]}
]
knn_best = trainer.get_best_model(knn_clf, params_grid)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:    0.2s


Best score: 0.7306823174941937
Configuration: {'n_neighbors': 6, 'weights': 'distance'}


[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    0.2s finished


In [109]:
final_model = rf_best

## Test

In [110]:
test = pd.read_csv('./data/test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [113]:
X_test = full_pipeline.transform(test)
y_pred = final_model.predict(X_test)

In [138]:
test_ans = test.copy()
test_ans['pred'] = y_pred
test_ans[test_ans.Sex == 'female'].head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,pred
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q,0
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C,1
12,904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S,1


In [115]:
submition = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submition.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,1


In [120]:
submition.to_csv('submition.csv', index=False)

In [121]:
!kaggle competitions submit -f submition.csv -m 'First try' titanic

100%|████████████████████████████████████████| 2.77k/2.77k [00:03<00:00, 746B/s]
Successfully submitted to Titanic: Machine Learning from Disaster